In [ ]:
from sym_so3 import *
from drake_sympy import *
import debug

Using the standard formulas:

Let $r = [r_r, r_p, r_y]$ be the Euler angles, and look at derivative values

\begin{align}
    \text{Skew symmetric angular velocity matrix:} \\
    \hat{\omega} &= R^T \dot{R}   \\
    \text{Skew symmetric angular acceleration matrix:} \\
    \hat{\alpha} &= \dot{R}^T \dot{R} + R^T \ddot{R} \\
        &= \hat{\omega}^T \hat{\omega} + R^T \ddot{R} \\
    \text{Alternatively, can use time derivative of (unskewed) angular velocity} \\
    \alpha &= \dot{\omega} = \frac{\partial \omega}{\partial r} \dot{r} + \frac{\partial \omega}{\partial \dot{r}} \ddot{r}
\end{align}

In [ ]:
from pydrake.symbolic import Jacobian

In [ ]:
workspace = SecondOrderWorkspace.make(["r", "p", "y"])

r_s, rd_s, rdd_s = workspace.syms

r_rd_s = cat(r_s, rd_s)
rd_rdd_s = cat(rd_s, rdd_s)

# Compute rotation matrix using rpy.
R_s = RollPitchYaw_[Expression](r_s).ToRotationMatrix().matrix()

# Compute first- and second-derivatives w.r.t. time.
Rd_s = derivatives_1st(R_s, r_s, rd_s)
Rd_s, Rdd_s = derivatives_1st_and_2nd(R_s, r_s, rd_s, rdd_s)
# Compute skew-symmetric angular velocity matrix.
wh_s = R_s.T @ Rd_s
w_s = unskew(wh_s, tol=None)

# Compute angular accel. directly from angular velocity.
ah_s = derivatives_1st(wh_s, r_rd_s, rd_rdd_s)

# Alternative
ah_alt_s = wh_s.T @ wh_s + R_s.T @ Rdd_s

In [ ]:
J_s = Jacobian(w_s, rd_s)

In [ ]:
env = workspace.env
r_v, rd_v, rdd_v = workspace.values

# # Single-axis: This shows numerical values that are OK for both angular accelerations.
# r_v[:] = [0.0, 0.0, 0.4]
# rd_v[:] = [0.0, 0.0, 0.7]
# rdd_v[:] = [0.0, 0.0, 1.1]

# Multi-axis: This does *not* show good numerical values, which is reflected in symbolics.
r_v[:] = [0.2, 0.3, 0.4]
rd_v[:] = [0.5, 0.6, 0.7]
rdd_v[:] = [0.8, 0.9, 1.1]

R = sym.Evaluate(R_s, env)
wh = sym.Evaluate(wh_s, env)
w = unskew(wh)

ah = sym.Evaluate(ah_s, env)
a = unskew(ah)
ah_alt = sym.Evaluate(ah_alt_s, env)
a_alt = unskew(ah_alt)

In [ ]:
# Decent-looking values.
print(R @ R.T)
print(w)
print(a)
print(a_alt)

In [ ]:
to_sympy = make_drake_to_sympy(cat(r_s, rd_s, rdd_s))
r_sympy = [to_sympy[hash(x)] for x in r_s]

def pretty(A, *, simplify=True):
    A = drake_to_sympy_matrix(A, to_sympy)
    A = pretty_trig(A, r_sympy, simplify=simplify)
    return A

The `trigsimp` stuff will take a few min

For compactness, I substitute stuff like $cos(r)$ with $c_r$, $sin(y)$ with $s_y$, etc.

In [ ]:
pretty(R_s)

In [ ]:
pretty(Rd_s)

In [ ]:
pretty(w_s)

In [ ]:
pretty(J_s)

In [ ]:
# Slow af
# pretty(Rdd_s)

In [ ]:
# pretty(ah_s)

In [ ]:
# pretty(ah_alt_s)

## Quaternions

In [ ]:
import copy

from pydrake.common.eigen_geometry import Quaternion_

In [ ]:
workspace = SecondOrderWorkspace.make(["w", "x", "y", "z"])
env = workspace.env
q_v, qd_v, qdd_v = workspace.values

q_s, qd_s, qdd_s = workspace.syms
q_qd_s = cat(q_s, qd_s)
qd_qdd_s = cat(qd_s, qdd_s)

q_norm_squared_s = (q_s ** 2).sum()
q_full_s = Quaternion_[Expression](wxyz=q_s)

def remove_q_norm_unit(expr):
    # Assume norm(q)^2 == 1 for expression.
    return drake_sym_replace(expr, q_norm_squared_s, 1.0)

R_s = RotationMatrix_[Expression](q_full_s).matrix()
R_s = remove_q_norm_unit(R_s)

Rd_s, Rdd_s = derivatives_1st_and_2nd(R_s, q_s, qd_s, qdd_s)
wh_s = R_s.T @ Rd_s
w_s = unskew(wh_s, tol=None)
ah_s = derivatives_1st(wh_s, q_qd_s, qd_qdd_s)
a_s = unskew(ah_s, tol=None)

In [ ]:
# Derivatives along normalization.
q_norm_s = q_s / np.sqrt(q_norm_squared_s)
qd_norm_s = derivatives_1st(q_norm_s, q_s, qd_s)
qd_norm_s = remove_q_norm_unit(qd_norm_s)
qdd_norm_s = derivatives_1st(qd_norm_s, q_qd_s, qd_qdd_s)
qdd_norm_s = remove_q_norm_unit(qdd_norm_s)

def normalize(x, *, tol=1e-8):
    n = np.linalg.norm(x)
    assert n > tol
    return x / n

def set_q_qd_qdd_norm(q_in, qd_in, qdd_in):
    # Normalize input values.
    # TODO(eric.cousineau): What about upper half-sphere?
    q_v[:] =  normalize(q_in)
    # Feed raw velocity, then re-evaluate.
    qd_v[:] = qd_in
    qd_v[:] = sym.Evaluate(qd_norm_s, env).reshape((-1,))
    # Feed raw accel, then re-evaluate.
    qdd_v[:] = qdd_in
    qdd_v[:] = sym.Evaluate(qdd_norm_s, env).reshape((-1,))

In [ ]:
# N.B. This is for *normalized* quaternion *and* time derivatives!
J_s = Jacobian(w_s, qd_s)

# As in Drake, should chain rule.
J_qd_norm_s = Jacobian(qd_norm_s, qd_s)

In [ ]:
to_sympy = make_drake_to_sympy(cat(q_s, qd_s, qdd_s))
q_sympy = [to_sympy[hash(x)] for x in q_s]

def pretty(A, *, simplify=True):
    A = drake_to_sympy_matrix(A, to_sympy)
    A = pretty_trig(A, q_sympy, simplify=simplify)
    return A

In [ ]:
pretty(R_s)

In [ ]:
pretty(Rd_s)

In [ ]:
pretty(Rdd_s)

In [ ]:
pretty(J_s)

In [ ]:
pretty(J_qd_norm_s)

In [ ]:
pretty(a_s)

In [ ]:
pretty(qd_norm_s)

In [ ]:
pretty(qdd_norm_s)

In [ ]:
set_q_qd_qdd_norm(
    q_in=[0.1, 0.2, 0.3, 0.4],
    qd_in=[0.5, 0.6, 0.7, 0.1],
    qdd_in=[10, 10, 10, 10],
)

In [ ]:
(q_v, qd_v, qdd_v)

In [ ]:
def allclose(a, b, *, tol=1e-10):
    return np.allclose(a, b, atol=tol, rtol=0.0)

In [ ]:
# The above *must* be a fixed point!!!
q_orig, qd_orig, qdd_orig = copy.deepcopy((q_v, qd_v, qdd_v))
set_q_qd_qdd_norm(q_orig, qd_orig, qdd_orig)
assert allclose(q_v, q_orig)
assert allclose(qd_v, qd_orig)
assert allclose(qdd_v, qdd_orig)

In [ ]:
R = sym.Evaluate(R_s, env)
wh = sym.Evaluate(wh_s, env)
w = unskew(wh)
ah = sym.Evaluate(ah_s, env)
a = unskew(ah)

print(R)
assert allclose(R @ R.T, np.eye(3))
print(w)
print(a)

## Exponential Coordinates

In [ ]:
# TODO(eric.cousineau): Fill this out.